In [8]:
import numpy as np
import h5py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pytorch_lightning import LightningModule
garstec_data = r'C:\Users\kiena\Documents\YEAR 4\PROJECT\Data\Garstec_AS09_chiara.hdf5'

In [7]:
# 7 Inputs
ages = []
massini = []
fehini = []
alphamlt = []
yini = []
eta = []
alphafe = []

# 5 Outputs
teff = []
luminosity = []
dnufit = []
FeH = []
numax = []

# Open the hdf5 file (read-only mode)
with h5py.File(garstec_data, 'r') as hdf:
    grid = hdf['grid']
    tracks = grid['tracks']

    # Get a list of track names and shuffle for random sampling
    track_names = list(tracks.keys())
    random.seed(1)
    random.shuffle(track_names)

    # Choose a subset of tracks to process (or not)
    selected_tracks = track_names[:]

    for track_name in selected_tracks:  # Iterate over the selected track names
        track = tracks[track_name]
        # Inputs
        ages.append(track['age'][:])
        massini.append(track['massini'][:])
        fehini.append(track['FeHini'][:])
        alphamlt.append(track['alphaMLT'][:])
        yini.append(track['yini'][:])
        eta.append(track['eta'][:])
        alphafe.append(track['alphaFe'][:])

        # Outputs
        teff.append(track['Teff'][:])
        luminosity.append(track['LPhot'][:])
        dnufit.append(track['dnufit'][:])
        FeH.append(track['FeH'][:])
        numax.append(track['numax'][:])

# Convert lists to numpy arrays and concatenate directly (no log transformation)
input_arrays = [
    np.concatenate(ages).reshape(-1, 1),
    np.concatenate(massini).reshape(-1, 1),
    np.concatenate(fehini).reshape(-1, 1),
    np.concatenate(alphamlt).reshape(-1, 1),
    np.concatenate(yini).reshape(-1, 1),
    np.concatenate(eta).reshape(-1, 1),
    np.concatenate(alphafe).reshape(-1, 1)
]

# Concatenate all inputs
inputs = np.hstack(input_arrays)

# Concatenate all outputs
output_arrays = [
    np.concatenate(teff).reshape(-1, 1),
    np.concatenate(luminosity).reshape(-1, 1),
    np.concatenate(dnufit).reshape(-1, 1),
    np.concatenate(numax).reshape(-1, 1),
    np.concatenate(FeH).reshape(-1, 1)
]

# Combine outputs
outputs = np.hstack(output_arrays)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.2, random_state=1)

# Scale data
input_scaler = StandardScaler()
output_scaler = StandardScaler()

X_train_scaled = input_scaler.fit_transform(X_train)
X_test_scaled = input_scaler.transform(X_test)

y_train_scaled = output_scaler.fit_transform(y_train)
y_test_scaled = output_scaler.transform(y_test)

# Store scaler parameters
output_scaler_mean = output_scaler.mean_
output_scaler_scale = output_scaler.scale_

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Assuming you have the y_train data loaded already as a numpy array
# If you don't have it, load it the same way you did during training
# y_train = ...

# Step 1: Refit the StandardScaler to the y_train data
output_scaler = StandardScaler()
output_scaler.fit(y_train)

# Step 2: Get the mean and scale (standard deviation) values
output_scaler_mean = output_scaler.mean_
output_scaler_scale = output_scaler.scale_

# Optionally, save them as .npy files for future use
np.save("output_scaler_mean.npy", output_scaler_mean)
np.save("output_scaler_scale.npy", output_scaler_scale)

# Now you can load them into your model (if needed later):
# output_scaler_mean = np.load("output_scaler_mean.npy")
# output_scaler_scale = np.load("output_scaler_scale.npy")


In [13]:
class GarstecNet(nn.Module):
    def __init__(self, input_dim, output_dim, loss_weights, output_scaler_mean, output_scaler_scale):
        super().__init__()
        self.loss_weights = torch.tensor(loss_weights)

        # Store scaler parameters as tensors
        self.register_buffer('output_mean', torch.tensor(output_scaler_mean, dtype=torch.float32))
        self.register_buffer('output_scale', torch.tensor(output_scaler_scale, dtype=torch.float32))

        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32, output_dim)
        )
    
    def inverse_transform(self, scaled_tensor):
        """Convert scaled outputs back to the original scale."""
        return scaled_tensor * self.output_scale + self.output_mean

    def forward(self, x):
        return self.model(x)
    
model = GarstecNet(7, 5, [70, 0.1, 0.1, 0.5/3090, 0.01], output_scaler_mean, output_scaler_scale)
model.load_state_dict(torch.load(best_model_path, map_location=torch.device("cpu"))["state_dict"])
model.eval()


GarstecNet(
  (model): Sequential(
    (0): Linear(in_features=7, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): LeakyReLU(negative_slope=0.01)
    (10): Linear(in_features=32, out_features=5, bias=True)
  )
)

In [14]:
# Convert to tensor
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# Get predictions (scaled)
y_pred_scaled = model(X_test_tensor).detach().cpu().numpy()

# Inverse transform predictions
y_pred_unscaled = model.inverse_transform(torch.tensor(y_pred_scaled)).detach().cpu().numpy()

# Display first few results
print("First 5 predictions (original scale):\n", y_pred_unscaled[:5])

RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1532672000 bytes.